In [8]:
""" Bisection method for the power utility u(x)=x**alpha in the case of 1 risky asset. 
(1) computes Table 1
(2) computes the optimal weight of the risky asset and its true utility, using large sample
"""

import numpy as np
from scipy  import optimize

In [9]:
# nu is the weignt of the risky asset
def g(nu,r):
    """ The derivative of the ordinary power utility """
    z=1+nu*(r-1)
    return np.mean(z**(alpha-1)*(r-1))
def gb(nu,r):
    """ The derivative of the relative power utility """
    z1=1+nu*(r-1)
    z2=np.fmax(r,1)
    return np.mean((z1/z2)**(alpha-1)*(r-1)/z2)

In [10]:
import random
# N is the sample length
n=252*10**3
# r: returns of the risky asset
r=np.ones(n)
# Parameters of the discrete Black-Scholes model
mu=0.15
sigma=0.45
print(mu,sigma)
print(np.exp(mu))
print((mu-sigma**2/2)/252,sigma/np.sqrt(252))

0.15 0.45
1.161834242728283
0.0001934523809523809 0.028347335475692043


In [11]:
# n_experiments is the number of realizations
n_experiments=100
# w_bis, wb_bis are the arrays for the optimal weights for the ordinary and relative power utilities
w_bis=np.zeros((n_experiments))
wb_bis=np.zeros((n_experiments))

In [12]:
w_al=[]
wb_al=[]
alphas=[0.001,0.01,0.1,0.2,0.3,0.5,0.75,0.9]
for alpha in alphas:
    for s in range(n_experiments):
        np.random.seed(42+s) 
      # returns of the risky asset
        r=np.random.lognormal(mean=(mu-sigma**2/2)/252, sigma=sigma/np.sqrt(252), size=n) 
      # The bisection method optimize.bisect for the derivatives of the empirical utilities with initial sign check
        if g(0,r)<=0 and g(1,r)<=0:
            root=0
        elif g(0,r)>=0 and g(1,r)>=0:
            root=1
        else: 
            root=optimize.bisect(g,0,1,args=(r)) 
        w_bis[s]=root
        if gb(0,r)<=0 and gb(1,r)<=0:
            rootb=0
        elif gb(0,r)>=0 and gb(1,r)>=0:
            rootb=1
        else: 
            rootb=optimize.bisect(gb,0,1,args=(r)) 
        wb_bis[s]=rootb
    # optimal weights of the risky asset averaged over n_experiments
    w_al.append(np.mean(w_bis))
    wb_al.append(np.mean(wb_bis))

In [13]:
# Table of optimal weights for ordinary and relative power utlity
import pandas as pd 
table=pd.DataFrame(index=['alpha','ordinary utility','relative utility'],data=np.round([alphas,w_al,wb_al],4))     
table

,0,1,2,3,4,5,6,7
alpha,0.0010,0.0100,0.1000,0.2000,0.3000,0.5000,0.7500,0.9
ordinary utility,0.7381,0.7448,0.8188,0.9118,0.9775,1.0000,1.0000,1.0
relative utility,0.7376,0.7397,0.7637,0.7961,0.8367,0.9245,0.9909,1.0


In [14]:

# This cell computes the optimal weight of the risky asset, using large ssample
N=10**7
alpha=0.2
np.random.seed(1000) 
r=np.random.lognormal(mean=(mu-sigma**2/2)/252, sigma=sigma/np.sqrt(252), size=N) 
if gb(0,r)<=0 and gb(1,r)<=0:
    rootb=0
elif gb(0,r)>=0 and gb(1,r)>=0:
    rootb=1
else: 
    rootb=optimize.bisect(gb,0,1,args=(r)) 
print(rootb)

0.8100864807911421


In [24]:
def U(nu,r):
    """ Power utility minus its value at zero
    """
    # r: the array of the risky asset returns
    z=((1+nu*(r-1))/np.fmax(r,1))**alpha
    return np.mean(z)

In [26]:
# The transformed utility of the optimal portfolio
U0=np.mean((1/np.fmax(r,1))**alpha)
(U(rootb,r)-U0)*10**4

0.4201594284181098